# 5.1 - Window Functions - Row Number

Neste notebook, vamos explorar as `Window Functions` (Funções de Janela) no PySpark e como utilizar row number para deduplicar dados 

## Imports

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/06 23:08:18 WARN Utils: Your hostname, MacBook-Air-de-Vitor.local, resolves to a loopback address: 127.0.0.1; using 192.168.3.49 instead (on interface en0)
26/02/06 23:08:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/06 23:08:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/06 23:08:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/06 23:08:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Para usar Window functions, precisamos importar Window do módulo `pyspark.sql.window`

In [2]:
from pyspark.sql.window import Window

## Deduplicando o dataset criado na ultima aula

In [7]:
path = "data/processed/join_features"

df = spark.read.parquet(path)
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- target: integer (nullable = true)
 |-- count_payment_value: long (nullable = true)
 |-- min_payment_value: double (nullable = true)
 |-- max_payment_value: double (nullable = true)
 |-- avg_payment_value: double (nullable = true)
 |-- sum_payment_value: double (nullable = true)
 |-- stddev_payment_value: double (nullable = true)
 |-- purchase_month: integer (nullable = true)
 |-- purchase_week: integer (nullable = true)
 |-- purchase_day_week: integer (nullable = true)
 |-- days_between: integer (nullable = true)



In [ ]:
# Verificando que o DataFrame está com o id duplicado

(
    df
    .groupBy("order_id")
    .count()
    .orderBy(F.col("count").desc())
).show(5)

+--------------------+-----+
|            order_id|count|
+--------------------+-----+
|df56136b8031ecd28...|    3|
|8e17072ec97ce29f0...|    3|
|03c939fd7fd3b38f8...|    3|
|c88b1d1b157a9999c...|    3|
|33f1e992ba3e439bf...|    2|
+--------------------+-----+
only showing top 5 rows


In [14]:
# Vamos pegar a pior nota de cada order_id
# Antes de aplicar no dataset final vamos utilizar um exemplo

data = [(1, 1), (1, 2), (2, 5), (2, 1)]
df_exemplo = spark.createDataFrame(data, ["id", "score"])
df_exemplo.show()

+---+-----+
| id|score|
+---+-----+
|  1|    1|
|  1|    2|
|  2|    5|
|  2|    1|
+---+-----+



In [ ]:
# Vamos ver como funciona o row_number ordenado pelo score
(
    df_exemplo
    .withColumn("rn", F.row_number().over(Window.orderBy("score")))
).show()

+---+-----+---+
| id|score| rn|
+---+-----+---+
|  1|    1|  1|
|  2|    1|  2|
|  1|    2|  3|
|  2|    5|  4|
+---+-----+---+



26/02/06 23:15:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/06 23:15:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/06 23:15:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [17]:
# Na ultima celula misturou os ids, para isso vamos utilizar partitionBy

(
    df_exemplo
    .withColumn("rn", F.row_number().over(Window.partitionBy("id").orderBy("score")))
).show()

# Agora para finalizar a deduplicação vamos filtrar rn = 1 

(
    df_exemplo
    .withColumn("rn", F.row_number().over(Window.partitionBy("id").orderBy("score")))
    .where(F.col("rn")==1)
).show()

+---+-----+---+
| id|score| rn|
+---+-----+---+
|  1|    1|  1|
|  1|    2|  2|
|  2|    1|  1|
|  2|    5|  2|
+---+-----+---+

+---+-----+---+
| id|score| rn|
+---+-----+---+
|  1|    1|  1|
|  2|    1|  1|
+---+-----+---+



## Aplicando no dataset de features e Salvando

In [ ]:
path = "data/processed/join_features_deduplicated"

w = Window.partitionBy("order_id").orderBy("review_score")

(
    df
    .withColumn("rn", F.row_number().over(w))
    .where(F.col("rn") == 1)
    .write
    .mode("overwrite")
    .parquet(path)
)